In [163]:
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import itertools     # 创建迭代器
import cvxpy as cp     # 寻找最小椭球体
from gensim.models import KeyedVectors     # 加载自己本地个性化的已训练好的词向量
from sklearn.decomposition import PCA     # 降维
from scipy.spatial.distance import pdist, squareform     # 用于计算欧几里得距离
from scipy.spatial import ConvexHull     # 用于找到向量点中的凸包，从而找出覆盖这些点的最小椭球体

In [148]:
text = pd.read_excel('narrative_structure.xlsx')
text

,词条,介绍
0,电子商务,电子商务，简称电商，是指在互联网或电子交易方式进行交易活动和相关服务活动，是传统商业活动各环...
1,网络直播,网络直播（英文：Livestream，又称推流）是指随着在线影音平台的兴起，在互联网上公开播...
2,短视频,短视频（英语：short video、video clip）又叫短片、小视频，是指长度至短1...
3,虚拟偶像,虚拟偶像（英语：virtual idol）是指虚构的、不存在于现实的偶像。在广泛的意义上，有...
4,体能锻炼,体能锻炼，又称体能训练、体适能训练，泛指所有通过运动方式，来达到维持与发展适当体能、增进身体...


In [45]:
# 切分原文档中每一条语料为句子
text_cut = []
for idx, row in text.iterrows():
    sentences = row['介绍'].replace('\\n', '').split('。')     # 按句号分割
    sentences = [s for s in sentences if s]  # 去除空句
    for i, sentence in enumerate(sentences, 1):     # 遍历每一句并添加到新的DataFrame
        text_cut.append({
            '词条': row['词条'],
            '句子编号': i,
            '句子': sentence
        })
df = pd.DataFrame(text_cut)
df

,词条,句子编号,句子
0,电子商务,1,电子商务，简称电商，是指在互联网或电子交易方式进行交易活动和相关服务活动，是传统商业活动各环...
1,电子商务,2,电子商务包括电子货币交换、供应链管理、电子交易市场、网络购物、网络营销、在线事务处理、电子数...
2,电子商务,3,在此过程中，利用到的信息技术包括互联网、万维网、电子邮件、数据库、电子目录和移动电话
3,电子商务,4,发展历史在过去的30年间，电子商务的概念发生了很大的变化
4,电子商务,5,最初，电子商务意味着利用电子化的手段，将商业买卖活动简化，通常使用的技术包括电子数据交换（E...
...,...,...,...
181,体能锻炼,69,因此运动类型、型态的选择及运动量皆因人而异
182,体能锻炼,70,过多的运动可能会导致女性的月经周期改变，引起月经关闭（俗称闭经）
183,体能锻炼,71,这满严重的，因为这意味着该女性运动者正在以不合理的方式使自己的体能超越原有的极限
184,体能锻炼,72,营养和收操适当的营养对于运动或维持健康来说都很重要，据2018年调查健身餐内容物，豆浆、牛奶...


## 语料预处理

### 剔除符号与数字

In [47]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['句子'][100]
remove_nums(test)

'随着智能移动端的用户规模不断地扩大让很多手机用户得以充分利用碎片时间拍摄观看短视频'

### 分词

In [49]:
# 加载中文停用词词典，可个性化设置
# stopwords = open('stopwords.txt', encoding = 'utf-8').read()

def clean_text(text):
    words = jieba.lcut(text)
#     words = [w for w in words if w not in stopwords]
    return ' '.join(words)
test = df['句子'][100]
clean_text(test)

'随着 智能 移动 端的 用户 规模 不断 地 扩大 ， 让 很多 手机用户 得以 充分利用 碎片 时间 拍摄 、 观看 短 视频'

### 预处理

In [51]:
df['分词'] = df['句子'].apply(remove_nums)
df['分词'] = df['分词'].apply(clean_text)
df['分词'] = df['分词'].apply(lambda x: x.split()).tolist()     # 切分词语
df.head()

,词条,句子编号,句子,分词
0,电子商务,1,电子商务，简称电商，是指在互联网或电子交易方式进行交易活动和相关服务活动，是传统商业活动各环...,"[电子商务, 简称, 电商, 是, 指, 在, 互联网, 或, 电子, 交易方式, 进行, ..."
1,电子商务,2,电子商务包括电子货币交换、供应链管理、电子交易市场、网络购物、网络营销、在线事务处理、电子数...,"[电子商务, 包括, 电子货币, 交换, 供应链, 管理, 电子, 交易市场, 网络, 购物..."
2,电子商务,3,在此过程中，利用到的信息技术包括互联网、万维网、电子邮件、数据库、电子目录和移动电话,"[在, 此, 过程, 中, 利用, 到, 的, 信息技术, 包括, 互联网, 万维网, 电子..."
3,电子商务,4,发展历史在过去的30年间，电子商务的概念发生了很大的变化,"[发展, 历史, 在, 过去, 的, 年间, 电子商务, 的, 概念, 发生, 了, 很大,..."
4,电子商务,5,最初，电子商务意味着利用电子化的手段，将商业买卖活动简化，通常使用的技术包括电子数据交换（E...,"[最初, 电子商务, 意味着, 利用, 电子化, 的, 手段, 将, 商业, 买卖, 活动,..."


## 划分文本区块（Chunk/Window） 

In [53]:
chunk = []
for entry, group in df.groupby('词条'):     # groupby函数返回值是对DataFrame的分组依据以及每个组包括的DataFrame段
    chunk_num = 1
    word_count = 0
    chunk_sentences = []
    chunk_words = []
    
    for idx, row in group.iterrows():
        sentence_words = row['分词']
        sentence_word_count = len(sentence_words)     # 数每个句子的词数
        
        if word_count + sentence_word_count < 250:     # 如果连续句子的词数相加低于一个chunk所要求的最低词数250，则继续相加
            word_count += sentence_word_count
            chunk_sentences.append(row['句子'])     # 拼接单个文本区块所包含的句子
            chunk_words.extend(sentence_words)     # 拼接单个文本区块所包含的句子的所有词语
        else:
            chunk.append({     # 如果连续句子的词数相加达到一个chunk所要求的最低词数250，则将该chunk导入语料库
                '词条': entry,
                '文本区块编号': chunk_num,
                '文本区块': chunk_sentences,
                '分词': chunk_words
            })

            chunk_num += 1     # 与此同时，开始新的文本区块
            word_count = sentence_word_count     # 使新一轮的文本区块词数基数为当前句子的词数
            chunk_sentences = [row['句子']]     # 使新一轮的文本区块句子为当前句子
            chunk_words = sentence_words

    chunk.append({     # 添加最后一个文本区块
        '词条': entry,
        '文本区块编号': chunk_num,
        '文本区块': chunk_sentences,
        '分词': chunk_words
    })

text_chunk = pd.DataFrame(chunk)
text_chunk

,词条,文本区块编号,文本区块,分词
0,体能锻炼,1,[体能锻炼，又称体能训练、体适能训练，泛指所有通过运动方式，来达到维持与发展适当体能、增进身...,"[体能, 锻炼, 又称, 体能训练, 体适, 能, 训练, 泛指, 所有, 通过, 运动, ..."
1,体能锻炼,2,[有规律运动且其运动强度大达到大约中、中强、强的程度的人，相对于没有规律运动习惯的人，可能有...,"[有, 规律, 运动, 且, 其, 运动, 强度, 大, 达到, 大约, 中中, 强强, 的..."
2,体能锻炼,3,[然而有些研究在运动选手选手身上发现，他们“长时间高强度的运动”与“淋巴细胞”的受迫率、“免...,"[然而, 有些, 研究, 在, 运动选手, 选手, 身上, 发现, 他们, 长时间, 高强度..."
3,体能锻炼,4,[ 大量的人体研究证明持之以恒的有氧运动（比如说：每天30分钟的有氧运动），脑部的认知功能将...,"[大量, 的, 人体, 研究, 证明, 持之以恒, 的, 有氧, 运动, 比如说, 每天, ..."
4,体能锻炼,5,[有适度规律地做有氧运动（比如跑步 、慢跑 、快步走、游泳和骑脚踏车等）的人通常在旨在测量一...,"[有, 适度, 规律, 地, 做, 有氧, 运动, 比如, 跑步, 慢跑, 快步, 走, 游..."
5,体能锻炼,6,[睡眠一个于2010年发布的系统性回顾表明：运动整体来说能提升绝大多数人的睡眠品质，并改善睡...,"[睡眠, 一个, 于, 年, 发布, 的, 系统性, 回顾, 表明, 运动, 整体, 来说,..."
6,体能锻炼,7,"[不适当的运动是弊大于利的，然而“适当”与否因人而异, 许多运动若未依照个人的体能状况做调节...","[不, 适当, 的, 运动, 是, 弊大于利, 的, 然而, 适当, 与否, 因人而异, 许..."
7,体能锻炼,8,[营养和收操适当的营养对于运动或维持健康来说都很重要，据2018年调查健身餐内容物，豆浆、牛...,"[营养, 和, 收操, 适当, 的, 营养, 对于, 运动, 或, 维持, 健康, 来说, ..."
8,电子商务,1,[电子商务，简称电商，是指在互联网或电子交易方式进行交易活动和相关服务活动，是传统商业活动各...,"[电子商务, 简称, 电商, 是, 指, 在, 互联网, 或, 电子, 交易方式, 进行, ..."
9,电子商务,2,"[接下来，在1998年和2000年之间，大量的美国和西欧公司开发了许多不成熟的网站, 在“....","[接下来, 在, 年, 和, 年, 之间, 大量, 的, 美国, 和, 西欧, 公司, 开发..."


## 词语向量化

In [57]:
# 加载本地下载好的预训练Word2Vec模型
wiki_model = KeyedVectors.load_word2vec_format('sgns.wiki.word', binary = False)

In [55]:
# 定义计算平均词向量的函数
def compute_average_vector(words, model):
    vectors = [model[word] for word in words if word in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.nan

In [58]:
# 计算平均词向量并添加到新的列中（文本区块）
text_chunk['平均词向量'] = text_chunk['分词'].apply(lambda x: compute_average_vector(x, wiki_model))

In [62]:
text_chunk

,词条,文本区块编号,文本区块,分词,平均词向量
0,体能锻炼,1,[体能锻炼，又称体能训练、体适能训练，泛指所有通过运动方式，来达到维持与发展适当体能、增进身...,"[体能, 锻炼, 又称, 体能训练, 体适, 能, 训练, 泛指, 所有, 通过, 运动, ...","[-0.058036063, -0.029137405, -0.16082147, -0.0..."
1,体能锻炼,2,[有规律运动且其运动强度大达到大约中、中强、强的程度的人，相对于没有规律运动习惯的人，可能有...,"[有, 规律, 运动, 且, 其, 运动, 强度, 大, 达到, 大约, 中中, 强强, 的...","[-0.073024884, -0.023917558, -0.12834004, -0.0..."
2,体能锻炼,3,[然而有些研究在运动选手选手身上发现，他们“长时间高强度的运动”与“淋巴细胞”的受迫率、“免...,"[然而, 有些, 研究, 在, 运动选手, 选手, 身上, 发现, 他们, 长时间, 高强度...","[-0.14029373, 0.0020581735, -0.12633392, 0.004..."
3,体能锻炼,4,[ 大量的人体研究证明持之以恒的有氧运动（比如说：每天30分钟的有氧运动），脑部的认知功能将...,"[大量, 的, 人体, 研究, 证明, 持之以恒, 的, 有氧, 运动, 比如说, 每天, ...","[-0.093076766, 0.036233883, -0.10762646, -0.05..."
4,体能锻炼,5,[有适度规律地做有氧运动（比如跑步 、慢跑 、快步走、游泳和骑脚踏车等）的人通常在旨在测量一...,"[有, 适度, 规律, 地, 做, 有氧, 运动, 比如, 跑步, 慢跑, 快步, 走, 游...","[-0.12016873, -0.0010756827, -0.11794597, -0.0..."
5,体能锻炼,6,[睡眠一个于2010年发布的系统性回顾表明：运动整体来说能提升绝大多数人的睡眠品质，并改善睡...,"[睡眠, 一个, 于, 年, 发布, 的, 系统性, 回顾, 表明, 运动, 整体, 来说,...","[-0.028097788, -0.028005274, -0.13563709, -0.0..."
6,体能锻炼,7,"[不适当的运动是弊大于利的，然而“适当”与否因人而异, 许多运动若未依照个人的体能状况做调节...","[不, 适当, 的, 运动, 是, 弊大于利, 的, 然而, 适当, 与否, 因人而异, 许...","[-0.025628168, 8.5718775e-06, -0.12657413, -0...."
7,体能锻炼,8,[营养和收操适当的营养对于运动或维持健康来说都很重要，据2018年调查健身餐内容物，豆浆、牛...,"[营养, 和, 收操, 适当, 的, 营养, 对于, 运动, 或, 维持, 健康, 来说, ...","[-0.0020798622, -0.0011082178, -0.17663138, -0..."
8,电子商务,1,[电子商务，简称电商，是指在互联网或电子交易方式进行交易活动和相关服务活动，是传统商业活动各...,"[电子商务, 简称, 电商, 是, 指, 在, 互联网, 或, 电子, 交易方式, 进行, ...","[-0.01312535, 0.063888654, -0.07535128, -0.103..."
9,电子商务,2,"[接下来，在1998年和2000年之间，大量的美国和西欧公司开发了许多不成熟的网站, 在“....","[接下来, 在, 年, 和, 年, 之间, 大量, 的, 美国, 和, 西欧, 公司, 开发...","[0.0059823096, 0.04307018, -0.08511054, -0.096..."


## 叙事结构

### 降维到三维空间

In [65]:
pca = PCA(n_components = 3)
average_vectors = np.array(list(text_chunk['平均词向量']))
text_chunk['降到三维后的平均词向量'] = list(pca.fit_transform(average_vectors))

In [68]:
text_chunk

,词条,文本区块编号,文本区块,分词,平均词向量,降到三维后的平均词向量
0,体能锻炼,1,[体能锻炼，又称体能训练、体适能训练，泛指所有通过运动方式，来达到维持与发展适当体能、增进身...,"[体能, 锻炼, 又称, 体能训练, 体适, 能, 训练, 泛指, 所有, 通过, 运动, ...","[-0.058036063, -0.029137405, -0.16082147, -0.0...","[0.60157055, -0.013906531, -0.054041427]"
1,体能锻炼,2,[有规律运动且其运动强度大达到大约中、中强、强的程度的人，相对于没有规律运动习惯的人，可能有...,"[有, 规律, 运动, 且, 其, 运动, 强度, 大, 达到, 大约, 中中, 强强, 的...","[-0.073024884, -0.023917558, -0.12834004, -0.0...","[0.6050824, 0.06907121, 0.02353532]"
2,体能锻炼,3,[然而有些研究在运动选手选手身上发现，他们“长时间高强度的运动”与“淋巴细胞”的受迫率、“免...,"[然而, 有些, 研究, 在, 运动选手, 选手, 身上, 发现, 他们, 长时间, 高强度...","[-0.14029373, 0.0020581735, -0.12633392, 0.004...","[0.6505152, -0.039320298, 0.039623488]"
3,体能锻炼,4,[ 大量的人体研究证明持之以恒的有氧运动（比如说：每天30分钟的有氧运动），脑部的认知功能将...,"[大量, 的, 人体, 研究, 证明, 持之以恒, 的, 有氧, 运动, 比如说, 每天, ...","[-0.093076766, 0.036233883, -0.10762646, -0.05...","[0.40457654, -0.022472665, -0.06730329]"
4,体能锻炼,5,[有适度规律地做有氧运动（比如跑步 、慢跑 、快步走、游泳和骑脚踏车等）的人通常在旨在测量一...,"[有, 适度, 规律, 地, 做, 有氧, 运动, 比如, 跑步, 慢跑, 快步, 走, 游...","[-0.12016873, -0.0010756827, -0.11794597, -0.0...","[0.5920339, -0.063303314, -0.032994833]"
5,体能锻炼,6,[睡眠一个于2010年发布的系统性回顾表明：运动整体来说能提升绝大多数人的睡眠品质，并改善睡...,"[睡眠, 一个, 于, 年, 发布, 的, 系统性, 回顾, 表明, 运动, 整体, 来说,...","[-0.028097788, -0.028005274, -0.13563709, -0.0...","[0.5235918, 0.019436706, 0.06468231]"
6,体能锻炼,7,"[不适当的运动是弊大于利的，然而“适当”与否因人而异, 许多运动若未依照个人的体能状况做调节...","[不, 适当, 的, 运动, 是, 弊大于利, 的, 然而, 适当, 与否, 因人而异, 许...","[-0.025628168, 8.5718775e-06, -0.12657413, -0....","[0.52363986, 0.026602749, 0.17785153]"
7,体能锻炼,8,[营养和收操适当的营养对于运动或维持健康来说都很重要，据2018年调查健身餐内容物，豆浆、牛...,"[营养, 和, 收操, 适当, 的, 营养, 对于, 运动, 或, 维持, 健康, 来说, ...","[-0.0020798622, -0.0011082178, -0.17663138, -0...","[0.48332968, -0.046189614, -0.11612704]"
8,电子商务,1,[电子商务，简称电商，是指在互联网或电子交易方式进行交易活动和相关服务活动，是传统商业活动各...,"[电子商务, 简称, 电商, 是, 指, 在, 互联网, 或, 电子, 交易方式, 进行, ...","[-0.01312535, 0.063888654, -0.07535128, -0.103...","[-0.35641128, 0.017052125, -0.2443446]"
9,电子商务,2,"[接下来，在1998年和2000年之间，大量的美国和西欧公司开发了许多不成熟的网站, 在“....","[接下来, 在, 年, 和, 年, 之间, 大量, 的, 美国, 和, 西欧, 公司, 开发...","[0.0059823096, 0.04307018, -0.08511054, -0.096...","[-0.3739416, 0.17071332, -0.2540433]"


### 叙事速度

按照Toubia(2021)对叙事速度的定义，叙事速度为整个文本从一个文本区块走到另一个文本区块的总路程除以总路径数。

In [71]:
# 定义函数计算两个点之间的欧几里得距离（就是向量范数）
def compute_euclidean_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

In [72]:
# 对每个词条计算相邻点的欧几里得距离的平均值
def compute_mean_distance(group):
    if len(group) == 1:     # 如果该词条下本就只有一个文本区块，则该文本的叙事速度计为缺失值
        return np.nan
    
    distances = [compute_euclidean_distance(group.iloc[i]['降到三维后的平均词向量'], group.iloc[i+1]['降到三维后的平均词向量']) 
                 for i in range(len(group)-1)]     # 逐个计算每个词条下第i和第i+1个点之间的距离，并保存到列表
    mean_distance = np.mean(distances)
    
    return mean_distance

In [137]:
# 对每个词条分组并应用上述函数
narrative_structure = text_chunk.groupby('词条').apply(compute_mean_distance).reset_index()
narrative_structure.columns = ['词条', '叙事速度']
narrative_structure

,词条,叙事速度
0,体能锻炼,0.179994
1,电子商务,0.285390
2,短视频,NaN
3,网络直播,0.071656
4,虚拟偶像,0.168989


### 叙事迂回度

按照Toubia(2021)对叙事迂回度的定义，叙事迂回度为整个文本在固定起始点和终点的情况下，遍历所有文本区块的实际总路程除以理论上的最短路径。这一问题其实是数学里面的经典“旅行商问题”。解决方案通常是动态规划，但由于我们的点不多，因此我们这里直接使用枚举法解决问题。

In [124]:
def enumerate_paths(start_point, end_point, points):
    all_points = [start_point] + points + [end_point]
    total_distance = float('inf')
    
    # 生成所有可能的点的排列
    for path in itertools.permutations(range(len(all_points))):
        if path[0] == 0 and path[-1] == len(all_points) - 1:
            distance = 0
            for i in range(len(path) - 1):
                distance += compute_euclidean_distance(all_points[path[i]], all_points[path[i+1]])
            
            total_distance = min(total_distance, distance)
    
    return total_distance

In [135]:
shortest_distance = []
for group_name, group_data in text_chunk.groupby('词条'):
    if len(group_data) == 1:
        shortest_distance.append(np.nan)
    elif len(group_data) == 2:
        start_point = group_data['降到三维后的平均词向量'][group_data['文本区块编号'] == 1].iloc[0]
        end_point = group_data['降到三维后的平均词向量'][group_data['文本区块编号'] == 2].iloc[0]
        shortest_distance.append(compute_euclidean_distance(start_point, end_point))
    else:
        points = group_data['降到三维后的平均词向量'].values.tolist()
        start_point = group_data['降到三维后的平均词向量'][group_data['文本区块编号'] == 1].iloc[0]
        end_point = group_data['降到三维后的平均词向量'][group_data['文本区块编号'] == max(group_data['文本区块编号'])].iloc[0]
        
        # 使用列表解析创建一个新的points列表，其中排除了end_point
        points = [point for point in points if not np.array_equal(point, end_point)]
        shortest_distance.append(enumerate_paths(start_point, end_point, points))

In [136]:
shortest_distance

[0.840653158724308, 2.013573043048382, nan, 0.071655914, 0.16898897]

In [140]:
actual_distance = []
for group_name, group_data in text_chunk.groupby('词条'):
    if len(group_data) == 1:     # 如果该词条下本就只有一个文本区块，则该文本的实际距离为缺失值
        actual_distance.append(np.nan)
    else:
        distances = [compute_euclidean_distance(group_data.iloc[i]['降到三维后的平均词向量'], group_data.iloc[i+1]['降到三维后的平均词向量']) 
                     for i in range(len(group_data)-1)]     # 逐个计算每个词条下第i和第i+1个点之间的距离，并保存到列表
        actual_distance.append(sum(distances))
actual_distance

[1.259956143796444,
 2.2831192687153816,
 nan,
 0.07165591418743134,
 0.16898897290229797]

In [141]:
narrative_structure['叙事迂回度'] = np.divide(actual_distance, shortest_distance)
narrative_structure

,词条,叙事速度,叙事迂回度
0,体能锻炼,0.179994,1.498782
1,电子商务,0.285390,1.133865
2,短视频,NaN,NaN
3,网络直播,0.071656,1.000000
4,虚拟偶像,0.168989,1.000000


### 叙事容量

按照Toubia(2021)对叙事容量的定义，叙事容量为整个文本覆盖所有向量的最小椭球体的体积。这个可以使用特定包通过解决优化问题来计算。

In [159]:
grouped_vectors = text_chunk.groupby('词条')['降到三维后的平均词向量'].apply(list).reset_index()

pretext = pd.DataFrame({
    '词条': grouped_vectors['词条'],
    '降到三维后的平均词向量': grouped_vectors['降到三维后的平均词向量']
})

text = pd.merge(text, pretext.drop_duplicates('词条')[['词条']], on='词条', how='left')
text

,词条,介绍,降到三维后的平均词向量
0,电子商务,电子商务，简称电商，是指在互联网或电子交易方式进行交易活动和相关服务活动，是传统商业活动各环...,"[[-0.35641128, 0.017052125, -0.2443446], [-0.3..."
1,网络直播,网络直播（英文：Livestream，又称推流）是指随着在线影音平台的兴起，在互联网上公开播...,"[[-0.50725234, -0.30801594, 0.08756051], [-0.4..."
2,短视频,短视频（英语：short video、video clip）又叫短片、小视频，是指长度至短1...,"[[-0.35379097, -0.22411513, 0.020264842]]"
3,虚拟偶像,虚拟偶像（英语：virtual idol）是指虚构的、不存在于现实的偶像。在广泛的意义上，有...,"[[-0.2544542, -0.5006306, 0.4843528], [-0.2542..."
4,体能锻炼,体能锻炼，又称体能训练、体适能训练，泛指所有通过运动方式，来达到维持与发展适当体能、增进身体...,"[[0.60157055, -0.013906531, -0.054041427], [0...."


In [179]:
volumes = []
for row in text['降到三维后的平均词向量']:
    if len(row) <= 2:  # 跳过少于3个点的行，因为不足3个点在空间当中无法形成立体结构
        volumes.append(np.nan)
        continue

    # 将数据转换为矩阵形式
    points = np.array(row).T

    # 创建优化变量
    A = cp.Variable((3, 3), symmetric=True)
    b = cp.Variable(3)
    r_square = cp.Variable()

    # 创建优化问题
    constraints = [cp.norm(A @ points[:, i] + b) <= r_square for i in range(points.shape[1])]  # 点在椭球体上
    constraints += [cp.trace(A) <= 3 * r_square]  # 确保椭球体是正定的
    constraints += [r_square >= 1e-6]  # 确保r_square非负
    objective = cp.Minimize(r_square)
    problem = cp.Problem(objective, constraints)

    # 求解优化问题
    problem.solve(solver=cp.SCS)

    # 保存体积
    if problem.status == 'optimal':
        volumes.append(np.sqrt(problem.value))
    else:
        volumes.append(np.nan)

In [180]:
volumes

[0.0013461174175745817, nan, nan, nan, 0.0013940636321104437]

In [182]:
narrative_structure['叙事容量'] = volumes
narrative_structure

,词条,叙事速度,叙事迂回度,叙事容量
0,体能锻炼,0.179994,1.498782,0.001346
1,电子商务,0.285390,1.133865,NaN
2,短视频,NaN,NaN,NaN
3,网络直播,0.071656,1.000000,NaN
4,虚拟偶像,0.168989,1.000000,0.001394


---------------------